In [101]:
import functions_framework
import smtplib, ssl

from email.mime.text import MIMEText
from datetime import datetime
from typing import Dict, Iterable
from google.cloud import compute_v1

# import os
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"]= "/Users/gil/git/ficc/creds.json" 

def list_all_instances(
    project_id: str,
) -> Dict[str, Iterable[compute_v1.Instance]]:
    """
    Returns a dictionary of all instances present in a project, grouped by their zone.

    Args:
        project_id: project ID or project number of the Cloud project you want to use.
    Returns:
        A dictionary with zone names as keys (in form of "zones/{zone_name}") and
        iterable collections of Instance objects as values.
    """
    instance_client = compute_v1.InstancesClient()
    request = compute_v1.AggregatedListInstancesRequest()
    request.project = project_id
    # Use the `max_results` parameter to limit the number of results that the API returns per response page.
    request.max_results = 50

    agg_list = instance_client.aggregated_list(request=request)
    instances = {}
    # Despite using the `max_results` parameter, you don't need to handle the pagination
    # yourself. The returned `AggregatedListPager` object handles pagination
    # automatically, returning separated pages as you iterate over the results.
    for zone, response in agg_list:
        if response.instances:
            #all_instances[zone] = response.instances
            for instance in response.instances:
                instances[instance.name] = {"status":instance.status,"machine_type":instance.machine_type,"last_start_timestamp":instance.last_start_timestamp,"last_stop_timestamp":instance.last_stop_timestamp,
                "zone":zone}
    return instances

def send_email(subject,error_message):
    receiver_email = "gil@ficc.ai"  #"ficcteam@ficc.ai"
    
    msg = MIMEText(error_message)
    msg['Subject'] = subject
    msg['From'] = "gil@ficc.ai"
    msg['To'] = receiver_email

    smtp_server = "smtp.gmail.com"
    port = 587
    sender_email = "notifications@ficc.ai"

    with smtplib.SMTP(smtp_server,port) as server:
        try:
            server.starttls()
            server.login(sender_email, 'ztwbwrzdqsucetbg')
            server.sendmail(sender_email, receiver_email, msg.as_string())
        except Exception as e:
            print(e)
        finally:
            server.quit() 

def main(cloudevent): 
    id = list_all_instances('eng-reactor-287421')
    running = [f"{k}, Status: {id[k]['status']}, last start time:{id[k]['last_start_timestamp'][:19]}, last stop time:{id[k]['last_stop_timestamp'][:19]} ({id[k]['machine_type']})" for k in id if id[k]['status'] == 'RUNNING']
    others = [f"{k}, Status: {id[k]['status']}, last start times:{id[k]['last_start_timestamp'][:19]}, last stop time:{id[k]['last_stop_timestamp'][:19]} ({id[k]['machine_type']})" for k in id if id[k]['status'] != 'RUNNING']
    running.insert(0,"\nRunning Instances:")
    others.insert(0,"\nOther Instances:")
    all = running+others

    vms = f"Compute Engine Summary for {datetime.now().strftime('%Y-%m-%d %H:%M')}\n"
    for vm_instance in all:
        vms+=vm_instance+"\n"
        vms+=cloudevent['Subject']
    send_email(f"Compute Engine Summary for {datetime.now().strftime('%Y-%m-%d %H:%M')}",vms)

# CloudEvent function to be triggered by an Eventarc Cloud Audit Logging trigger
# Note: this is NOT designed for second-party (Cloud Audit Logs -> Pub/Sub) triggers!
@functions_framework.cloud_event
def hello_auditlog(cloudevent):
   # Print out the CloudEvent's (required) `type` property
   # See https://github.com/cloudevents/spec/blob/v1.0.1/spec.md#type
   print(f"Event type: {cloudevent['type']}")

   # Print out the CloudEvent's (optional) `subject` property
   # See https://github.com/cloudevents/spec/blob/v1.0.1/spec.md#subject
   if 'subject' in cloudevent:
       # CloudEvent objects don't support `get` operations.
       # Use the `in` operator to verify `subject` is present.
       print(f"Subject: {cloudevent['subject']}")

   # Print out details from the `protoPayload`
   # This field encapsulates a Cloud Audit Logging entry
   # See https://cloud.google.com/logging/docs/audit#audit_log_entry_structure

   payload = cloudevent.data.get("protoPayload")
   if payload:
       print(f"API method: {payload.get('methodName')}")
       print(f"Resource name: {payload.get('resourceName')}")
       print(f"Principal: {payload.get('authenticationInfo', dict()).get('principalEmail')}")


In [102]:
main('f')